In [3]:
!pip install ortools

  Using cached ortools-9.10.4067-cp311-cp311-win_amd64.whl.metadata (3.0 kB)
  Using cached protobuf-5.27.1-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached immutabledict-4.2.0-py3-none-any.whl.metadata (3.4 kB)
   ---------------------------------------- 0.0/130.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/130.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/130.2 MB 217.9 kB/s eta 0:09:58
   ---------------------------------------- 0.0/130.2 MB 217.9 kB/s eta 0:09:58
   ---------------------------------------- 0.0/130.2 MB 145.2 kB/s eta 0:14:57
   ---------------------------------------- 0.0/130.2 MB 145.2 kB/s eta 0:14:57
   ---------------------------------------- 0.0/130.2 MB 145.2 kB/s eta 0:14:57
   ---------------------------------------- 0.0/130.2 MB 122.9 kB/s eta 0:17:40
   ---------------------------------------- 0.0/130.2 MB 122.9 kB/s eta 0:17:40
   ---------------------------------------- 0.0/130.2 MB 122.9 kB/s eta 0:

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
googleapis-common-protos 1.63.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-api-core 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-language 2.13.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
google-cloud-translate 3.15.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.27.1 which is incompatible.
proto-plus 1.23.0 requires protobuf<5.0.0dev,>=3.19.0, but you have prot

   --- ----------------------------------- 12.2/130.2 MB 337.1 kB/s eta 0:05:51
   --- ----------------------------------- 12.2/130.2 MB 335.7 kB/s eta 0:05:52
   --- ----------------------------------- 12.2/130.2 MB 335.7 kB/s eta 0:05:52
   --- ----------------------------------- 12.2/130.2 MB 335.4 kB/s eta 0:05:52
   --- ----------------------------------- 12.2/130.2 MB 334.3 kB/s eta 0:05:53
   --- ----------------------------------- 12.2/130.2 MB 334.9 kB/s eta 0:05:53
   --- ----------------------------------- 12.2/130.2 MB 334.9 kB/s eta 0:05:53
   --- ----------------------------------- 12.2/130.2 MB 334.9 kB/s eta 0:05:53
   --- ----------------------------------- 12.3/130.2 MB 333.2 kB/s eta 0:05:55
   --- ----------------------------------- 12.3/130.2 MB 334.9 kB/s eta 0:05:53
   --- ----------------------------------- 12.4/130.2 MB 336.0 kB/s eta 0:05:51
   --- ----------------------------------- 12.4/130.2 MB 336.6 kB/s eta 0:05:51
   --- ---------------------------------

In [2]:
pip show ortools

^C
Note: you may need to restart the kernel to use updated packages.
Name: ortools
Version: 9.10.4067
Summary: Google OR-Tools python libraries and modules
Home-page: https://developers.google.com/optimization/
Author: Google LLC
Author-email: or-tools@google.com
License: Apache 2.0
Location: C:\Users\asus\anaconda3\Lib\site-packages
Requires: absl-py, immutabledict, numpy, pandas, protobuf
Required-by: 


In [ ]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

# Function to read the distance matrix from an Excel file
def read_distance_matrix(city):
    destination_filepath = f'{city.lower()}_distance_duration_matrices.xlsx'  
    hotel_filepath = 'hotel_destination.xlsx'# Path to your Excel file
    df = pd.read_excel(destination_filepath, sheet_name='Distance Matrix')
    df.columns = ['Source', 'Destination', 'Distance (meters)']
    df['Type'] = 'Tourist Attraction'
    df_hotel  = pd.read_excel(hotel_filepath, sheet_name=city)
    df_hotel = df_hotel[['origin', 'destination', 'distance']]
    df_hotel['Type'] = 'Hotel'
    df = pd.concat([df, df_hotel], ignore_index=True)
    
    unique_destinations = pd.unique(df[['Source', 'Destination']].values.ravel('K'))
    destination_index = {destination: index for index, destination in enumerate(unique_destinations)}
    
    distance_matrix = [[0] * len(unique_destinations) for _ in range(len(unique_destinations))]
    
    for _, row in df.iterrows():
        source_index = destination_index[row['Source']]
        dest_index = destination_index[row['Destination']]
        distance_matrix[source_index][dest_index] = row['Distance (meters)']
    
    return distance_matrix, destination_index, unique_destinations

# Function to read destination prices from the dataframe
def read_destination_prices(city, df):
    prices = df[df['City'] == city]['Price'].tolist()
    return prices

# Function to read average time spent at each destination from the dataframe
def read_avg_time_spent(city, df):
    avg_time = df[df['City'] == city]['Time_Minutes'].fillna(60).tolist()
    return avg_time


In [26]:
# Function to create the data model
def create_data_model(city, price, destination_limit_per_day, num_days, num_vehicles=1, depot=0):
    df = pd.read_excel("indonesia_tourism.xlsx", sheet_name="indonesia_tourism")
    distance_matrix, destination_index, unique_destinations = read_distance_matrix(city)
    data = {}
    data['distance_matrix'] = distance_matrix
    data['prices'] = read_destination_prices(city, df)
    data['average_spent_time'] = read_avg_time_spent(city, df)
    data['price_threshold'] = price
    data['destination_limit_per_day'] = destination_limit_per_day
    data['num_days'] = num_days
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    data['place_names'] = unique_destinations  # Store the place names
    data['destination_index'] = destination_index  # Store the destination index
    
    # Debug print
    print("Distance Matrix:")
    for row in data['distance_matrix']:
        print(row)
        
    return data


In [27]:
data = create_data_model('Surabaya', 3000000, 4, 3)

Distance Matrix:
[0.0, 10359.0, 16290.0, 16288.0, 15115.0, 5640.0, 13032.0, 13008.0, 10237.0, 14349.0, 36591.0, 35397.0, 14798.0, 38238.0, 5286.0, 16819.0, 13280.0, 17311.0, 36589.0, 16923.0, 19785.0, 33222.0, 19863.0, 20950.0, 16284.0, 16191.0, 19198.0, 19198.0, 17515.0, 19607.0, 23809.0, 13308.0, 15613.0, 15726.0, 16531.0, 16309.0, 16011.0, 21100.0, 15491.0, 18306.0, 17352.0, 25422.0, 12820.0, 16290.0, 9858.0, 20884.0, 0]
[9514.0, 0.0, 7754.0, 10389.0, 9217.0, 6426.0, 3817.0, 11180.0, 122.0, 8450.0, 19133.0, 15884.0, 5735.0, 18725.0, 5842.0, 10920.0, 9252.0, 11380.0, 36068.0, 11024.0, 19265.0, 17273.0, 13964.0, 15052.0, 10385.0, 7127.0, 13299.0, 13299.0, 11617.0, 13708.0, 17910.0, 9279.0, 6549.0, 6663.0, 10633.0, 7246.0, 10113.0, 15201.0, 6427.0, 12407.0, 11453.0, 24902.0, 8562.0, 7754.0, 6598.0, 14985.0, 0]
[15468.0, 7458.0, 0.0, 10292.0, 9119.0, 12380.0, 4258.0, 17156.0, 7336.0, 10174.0, 21185.0, 12046.0, 2276.0, 14888.0, 11797.0, 9987.0, 13632.0, 14457.0, 26420.0, 9259.0, 25219.0,

In [29]:
def print_solution(manager, routing, solution, data):
    """Prints solution on console."""
    total_distance = 0
    total_price = 0
    total_time = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        route_distance = 0
        route_price = 0
        route_time = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            destination_name = data['place_names'][node_index]
            route_price += data['prices'][node_index]
            route_time += data['average_spent_time'][node_index]
            plan_output += f' {destination_name} -> '
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        plan_output += f' {data["place_names"][manager.IndexToNode(index)]}\n'
        plan_output += f'Distance of the route: {route_distance}m\n'
        plan_output += f'Price of the route: {route_price}\n'
        plan_output += f'Time of the route: {route_time} minutes\n'
        print(plan_output)
        total_distance += route_distance
        total_price += route_price
        total_time += route_time
    print(f'Total Distance of all routes: {total_distance}m')
    print(f'Total Price of all routes: {total_price}')
    print(f'Total Time of all routes: {int(total_time)} minutes')


In [30]:
def main():
    # Example usage
    city = 'Surabaya'
    price_threshold = 500000
    destination_limit_per_day = 4
    num_days = 3

    data = create_data_model(city, price_threshold, destination_limit_per_day, num_days)

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        100000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        'Distance')

    # Add Price constraint.
    def price_callback(from_index):
        """Returns the price of the node."""
        from_node = manager.IndexToNode(from_index)
        return data['prices'][from_node]

    price_callback_index = routing.RegisterUnaryTransitCallback(price_callback)
    routing.AddDimensionWithVehicleCapacity(
        price_callback_index,
        0,  # no slack
        [data['price_threshold']] * data['num_vehicles'],  # vehicle maximum price capacity
        True,  # start cumul to zero
        'Price')

    # Add Time constraint.
    def time_callback(from_index):
        """Returns the time spent at the node."""
        from_node = manager.IndexToNode(from_index)
        return data['average_spent_time'][from_node]

    time_callback_index = routing.RegisterUnaryTransitCallback(time_callback)
    routing.AddDimension(
        time_callback_index,
        0,  # no slack
        8 * 60,  # 8 hours in minutes
        True,  # start cumul to zero
        'Time')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(manager, routing, solution, data)
    else:
        print('No solution found !')

if __name__ == '__main__':
    main()

Distance Matrix:
[0.0, 10359.0, 16290.0, 16288.0, 15115.0, 5640.0, 13032.0, 13008.0, 10237.0, 14349.0, 36591.0, 35397.0, 14798.0, 38238.0, 5286.0, 16819.0, 13280.0, 17311.0, 36589.0, 16923.0, 19785.0, 33222.0, 19863.0, 20950.0, 16284.0, 16191.0, 19198.0, 19198.0, 17515.0, 19607.0, 23809.0, 13308.0, 15613.0, 15726.0, 16531.0, 16309.0, 16011.0, 21100.0, 15491.0, 18306.0, 17352.0, 25422.0, 12820.0, 16290.0, 9858.0, 20884.0, 0]
[9514.0, 0.0, 7754.0, 10389.0, 9217.0, 6426.0, 3817.0, 11180.0, 122.0, 8450.0, 19133.0, 15884.0, 5735.0, 18725.0, 5842.0, 10920.0, 9252.0, 11380.0, 36068.0, 11024.0, 19265.0, 17273.0, 13964.0, 15052.0, 10385.0, 7127.0, 13299.0, 13299.0, 11617.0, 13708.0, 17910.0, 9279.0, 6549.0, 6663.0, 10633.0, 7246.0, 10113.0, 15201.0, 6427.0, 12407.0, 11453.0, 24902.0, 8562.0, 7754.0, 6598.0, 14985.0, 0]
[15468.0, 7458.0, 0.0, 10292.0, 9119.0, 12380.0, 4258.0, 17156.0, 7336.0, 10174.0, 21185.0, 12046.0, 2276.0, 14888.0, 11797.0, 9987.0, 13632.0, 14457.0, 26420.0, 9259.0, 25219.0,

IndexError: list index out of range

In [82]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def read_distance_matrix(city):
    destination_filepath = f'{city.lower()}_distance_duration_matrices.xlsx'
    df = pd.read_excel(destination_filepath, sheet_name='Distance Matrix')
    df.columns = ['Source', 'Destination', 'Distance (meters)']
    df['Type'] = 'Tourist Attraction'
    
    unique_destinations = pd.unique(df[['Source', 'Destination']].values.ravel('K'))
    destination_index = {destination: index for index, destination in enumerate(unique_destinations)}
    
    distance_matrix = [[0] * len(unique_destinations) for _ in range(len(unique_destinations))]
    
    for _, row in df.iterrows():
        source_index = destination_index[row['Source']]
        dest_index = destination_index[row['Destination']]
        distance_matrix[source_index][dest_index] = row['Distance (meters)']
    
    return distance_matrix, destination_index, unique_destinations

def read_destination_prices(city, df):
    return df['Price'].tolist()

def read_avg_time_spent(city, df):
    return df['Time_Minutes'].tolist()

def create_data_model(city, price, destination_limit_per_day, num_days, num_vehicles=1, depot=0):
    df = pd.read_excel("indonesia_tourism.xlsx", sheet_name="indonesia_tourism")
    distance_matrix, destination_index, unique_destinations = read_distance_matrix(city)
    
    data = {}
    data['distance_matrix'] = distance_matrix
    data['prices'] = read_destination_prices(city, df)
    data['average_spent_time'] = read_avg_time_spent(city, df)
    data['price_threshold'] = price
    data['destination_limit_per_day'] = destination_limit_per_day
    data['num_days'] = num_days
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    data['place_names'] = unique_destinations
    data['destination_index'] = destination_index
    data['city'] = city
    
    return data

def select_hotel_within_budget(city, remaining_budget, hotels):
    valid_hotels = hotels[hotels['price'] <= remaining_budget]
    if valid_hotels.empty:
        return None
    return valid_hotels.loc[valid_hotels['price'].idxmin(), 'name']

def main():
    city = 'Surabaya'
    price_threshold = 2000000  # Increased price threshold
    destination_limit_per_day = 5  # Maximum destinations per day
    num_days = 1
    
    data = create_data_model(city, price_threshold, destination_limit_per_day, num_days)
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def cost_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        return distance
    
    transit_callback_index = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        [data['price_threshold']] * data['num_vehicles'],  # vehicle maximum price capacity
        True,  # start cumul to zero
        'Price')
    
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        [8 * 60] * data['num_vehicles'],  # vehicle maximum time capacity per day (480 minutes)
        True,  # start cumul to zero
        'Time')
    
    def print_solution(manager, routing, solution, data):
        total_distance = 0
        total_price = 0
        total_time = 0
        for day in range(data['num_days']):
            print(f"Day {day + 1} Route:")
            if solution[day]:
                index = routing.Start(day)
                plan_output = f'Route for vehicle {day}:\n'
                route_distance = 0
                route_price = 0
                route_time = 0

                remaining_budget = data['price_threshold']
                hotels = pd.read_excel(f'top_{data["city"].lower()}_hotel.xlsx')
                start_hotel = select_hotel_within_budget(data["city"].lower(), remaining_budget, hotels)
                end_hotel = start_hotel

                plan_output += f'Starting Hotel: {start_hotel}\n'
                num_destinations = 0

                while not routing.IsEnd(index):
                    node_index = manager.IndexToNode(index)
                    destination_name = data['place_names'][node_index]

                    next_index = solution[day].Value(routing.NextVar(index))

                    # Fetch arc cost for distance
                    arc_distance = routing.GetArcCostForVehicle(index, next_index, day)
                    route_distance += arc_distance

                    # Fetch arc cost for price
                    arc_price = routing.GetArcCostForVehicle(index, next_index, 'Price')
                    route_price += arc_price

                    # Fetch arc cost for time
                    arc_time = routing.GetArcCostForVehicle(index, next_index, 'Time')
                    route_time += arc_time

                    plan_output += f' {destination_name} -> '
                    index = next_index
                    num_destinations += 1
                
                plan_output += f' {data["place_names"][manager.IndexToNode(index)]}\n'
                plan_output += f'Ending Hotel: {end_hotel}\n'

                route_distance += 2 * data['distance_matrix'][0][manager.IndexToNode(index)]  # Round trip to depot

                plan_output += f'Distance of the route: {route_distance}m\n'
                plan_output += f'Price of the route: {route_price}\n'
                plan_output += f'Time of the route: {route_time} minutes\n'
                print(plan_output)

                total_distance += route_distance
                total_price += route_price
                total_time += route_time

                print(f"Number of destinations for Day {day + 1}: {num_destinations}\n")
            else:
                print(f"No solution found for Day {day + 1}!\n")

        print(f'Total Distance of all routes: {total_distance}m')
        print(f'Total Price of all routes: {total_price}')
        print(f'Total Time of all routes: {int(total_time)} minutes')

    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = []
    for day in range(num_days):
        solution.append(routing.SolveWithParameters(search_parameters))
    
    print_solution(manager, routing, solution, data)
    
    # Debug prints
    print("\nDebug Information:")
    print("Number of Nodes:", manager.GetNumberOfNodes())
    print("Number of Vehicles:", data['num_vehicles'])
    print("Price Capacity per Vehicle:", data['price_threshold'])
    print("Time Capacity per Vehicle per Day:", 12 * 60)  # This is per day (480 minutes)
    print("City:", data['city'])

if __name__ == '__main__':
    main()


Day 1 Route:
No solution found for Day 1!

Total Distance of all routes: 0m
Total Price of all routes: 0
Total Time of all routes: 0 minutes

Debug Information:
Number of Nodes: 46
Number of Vehicles: 1
Price Capacity per Vehicle: 2000000
Time Capacity per Vehicle per Day: 720
City: Surabaya


In [ ]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def read_distance_matrix(city):
    destination_filepath = f'{city.lower()}_distance_duration_matrices.xlsx'
    df = pd.read_excel(destination_filepath, sheet_name='Distance Matrix')
    df.columns = ['Source', 'Destination', 'Distance (meters)']
    df['Type'] = 'Tourist Attraction'
    
    unique_destinations = pd.unique(df[['Source', 'Destination']].values.ravel('K'))
    destination_index = {destination: index for index, destination in enumerate(unique_destinations)}
    
    distance_matrix = [[0] * len(unique_destinations) for _ in range(len(unique_destinations))]
    
    for _, row in df.iterrows():
        source_index = destination_index[row['Source']]
        dest_index = destination_index[row['Destination']]
        distance_matrix[source_index][dest_index] = row['Distance (meters)']
    
    return distance_matrix, destination_index, unique_destinations

def read_destination_prices(city, df):
    return df['Price'].tolist()

def read_avg_time_spent(city, df):
    return df['Time_Minutes'].tolist()

def create_data_model(city, price, destination_limit_per_day, num_days, num_vehicles=1, depot=0):
    df = pd.read_excel("indonesia_tourism.xlsx", sheet_name="indonesia_tourism")
    distance_matrix, destination_index, unique_destinations = read_distance_matrix(city)
    
    data = {}
    data['distance_matrix'] = distance_matrix
    data['prices'] = read_destination_prices(city, df)
    data['average_spent_time'] = read_avg_time_spent(city, df)
    data['price_threshold'] = price
    data['destination_limit_per_day'] = destination_limit_per_day
    data['num_days'] = num_days
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    data['place_names'] = unique_destinations
    data['destination_index'] = destination_index
    data['city'] = city
    
    return data

def select_hotel_within_budget(city, remaining_budget, hotels):
    valid_hotels = hotels[hotels['price'] <= remaining_budget]
    if valid_hotels.empty:
        return None
    return valid_hotels.loc[valid_hotels['price'].idxmin(), 'name']

def main():
    city = 'Surabaya'
    price_threshold = 5000000
    destination_limit_per_day = 4
    num_days = 3
    
    data = create_data_model(city, price_threshold, destination_limit_per_day, num_days)
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def cost_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        return distance
    
    transit_callback_index = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        [data['price_threshold']] * data['num_vehicles'],  # vehicle maximum price capacity
        True,  # start cumul to zero
        'Price')
    
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        [24 * 60 * data['destination_limit_per_day']] * data['num_vehicles'],  # vehicle maximum time capacity
        True,  # start cumul to zero
        'Time')
    
    def print_solution(manager, routing, solution, data):
        total_distance = 0
        total_price = 0
        total_time = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = f'Route for vehicle {vehicle_id}:\n'
            route_distance = 0
            route_price = 0
            route_time = 0
            
            remaining_budget = data['price_threshold']
            hotels = pd.read_excel(f'top_{data["city"].lower()}_hotel.xlsx')
            start_hotel = select_hotel_within_budget(data["city"].lower(), remaining_budget, hotels)
            end_hotel = start_hotel
            
            plan_output += f'Starting Hotel: {start_hotel}\n'
            
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                destination_name = data['place_names'][node_index]
                
                next_index = solution.Value(routing.NextVar(index))
                
                # Fetch arc cost for distance
                arc_distance = routing.GetArcCostForVehicle(index, next_index, vehicle_id)
                route_distance += arc_distance
                
                # Fetch arc cost for price
                arc_price = routing.GetArcCostForVehicle(index, next_index, 'Price')
                route_price += arc_price
                
                # Fetch arc cost for time
                arc_time = routing.GetArcCostForVehicle(index, next_index, 'Time')
                route_time += arc_time
                
                plan_output += f' {destination_name} -> '
                index = next_index
            
            plan_output += f' {data["place_names"][manager.IndexToNode(index)]}\n'
            plan_output += f'Ending Hotel: {end_hotel}\n'
            
            route_distance += 2 * data['distance_matrix'][0][manager.IndexToNode(index)]  # Round trip to depot
            
            plan_output += f'Distance of the route: {route_distance}m\n'
            plan_output += f'Price of the route: {route_price}\n'
            plan_output += f'Time of the route: {route_time} minutes\n'
            print(plan_output)
            
            total_distance += route_distance
            total_price += route_price
            total_time += route_time
        
        print(f'Total Distance of all routes: {total_distance}m')
        print(f'Total Price of all routes: {total_price}')
        print(f'Total Time of all routes: {int(total_time)} minutes')

    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(manager, routing, solution, data)
    else:
        print('No solution found!')
    
    # Debug prints
    print("Debug Information:")
    print("Number of Nodes:", manager.GetNumberOfNodes())
    print("Number of Vehicles:", data['num_vehicles'])
    print("Price Capacity per Vehicle:", data['price_threshold'])
    print("Time Capacity per Vehicle:", 24 * 60 * data['destination_limit_per_day'])
    print("City:", data['city'])

if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def read_distance_matrix(city):
    destination_filepath = f'{city.lower()}_distance_duration_matrices.xlsx'
    df = pd.read_excel(destination_filepath, sheet_name='Distance Matrix')
    df.columns = ['Source', 'Destination', 'Distance (meters)']
    df['Type'] = 'Tourist Attraction'
    
    unique_destinations = pd.unique(df[['Source', 'Destination']].values.ravel('K'))
    destination_index = {destination: index for index, destination in enumerate(unique_destinations)}
    
    distance_matrix = [[0] * len(unique_destinations) for _ in range(len(unique_destinations))]
    
    for _, row in df.iterrows():
        source_index = destination_index[row['Source']]
        dest_index = destination_index[row['Destination']]
        distance_matrix[source_index][dest_index] = row['Distance (meters)']
    
    return distance_matrix, destination_index, unique_destinations

def read_destination_prices(city, df):
    return df['Price'].tolist()

def read_avg_time_spent(city, df):
    return df['Time_Minutes'].tolist()

def create_data_model(city, price, destination_limit_per_day, num_days, num_vehicles=1, depot=0):
    df = pd.read_excel("indonesia_tourism.xlsx", sheet_name="indonesia_tourism")
    distance_matrix, destination_index, unique_destinations = read_distance_matrix(city)
    
    data = {}
    data['distance_matrix'] = distance_matrix
    data['prices'] = read_destination_prices(city, df)
    data['average_spent_time'] = read_avg_time_spent(city, df)
    data['price_threshold'] = price
    data['destination_limit_per_day'] = destination_limit_per_day
    data['num_days'] = num_days
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    data['place_names'] = unique_destinations
    data['destination_index'] = destination_index
    data['city'] = city
    
    return data

def select_hotel_within_budget(city, remaining_budget, hotels):
    valid_hotels = hotels[hotels['price'] <= remaining_budget]
    if valid_hotels.empty:
        return None
    return valid_hotels.loc[valid_hotels['price'].idxmin(), 'name']

def main():
    city = 'Surabaya'
    price_threshold = 500000
    destination_limit_per_day = 4
    num_days = 3
    
    data = create_data_model(city, price_threshold, destination_limit_per_day, num_days)
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def cost_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        distance = data['distance_matrix'][from_node][to_node]
        price = data['prices'][to_node]
        time = data['average_spent_time'][to_node]
        return distance, price, time
    
    transit_callback_index = routing.RegisterTransitCallback(cost_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    routing.AddDimensionWithVehicleCapacity(
        transit_callback_index,
        0,  # no slack
        [data['price_threshold']] * data['num_vehicles'],  # vehicle maximum price capacity
        True,  # start cumul to zero
        'Price')
    
    def print_solution(manager, routing, solution, data):
        total_distance = 0
        total_price = 0
        total_time = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            plan_output = f'Route for vehicle {vehicle_id}:\n'
            route_distance = 0
            route_price = 0
            route_time = 0
            
            remaining_budget = data['price_threshold']
            hotels = pd.read_excel(f'top_{data["city"].lower()}_hotel.xlsx')
            start_hotel = select_hotel_within_budget(data["city"].lower(), remaining_budget, hotels)
            end_hotel = start_hotel
            
            plan_output += f'Starting Hotel: {start_hotel}\n'
            
            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                destination_name = data['place_names'][node_index]
                
                next_index = solution.Value(routing.NextVar(index))
                arc_cost = routing.GetArcCostForVehicle(index, next_index, vehicle_id)
                
                route_distance += arc_cost  # Assuming this directly gives distance
                route_price += arc_cost   # Assuming this directly gives price
                route_time += arc_cost   # Assuming this directly gives time
                
                plan_output += f' {destination_name} -> '
                index = next_index
            
            plan_output += f' {data["place_names"][manager.IndexToNode(index)]}\n'
            plan_output += f'Ending Hotel: {end_hotel}\n'
            
            route_distance += 2 * data['distance_matrix'][0][manager.IndexToNode(index)]  # Round trip to depot
            
            plan_output += f'Distance of the route: {route_distance}m\n'
            plan_output += f'Price of the route: {route_price}\n'
            plan_output += f'Time of the route: {route_time} minutes\n'
            print(plan_output)
            
            total_distance += route_distance
            total_price += route_price
            total_time += route_time
        
        print(f'Total Distance of all routes: {total_distance}m')
        print(f'Total Price of all routes: {total_price}')
        print(f'Total Time of all routes: {int(total_time)} minutes')

    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    if solution:
        print_solution(manager, routing, solution, data)
    else:
        print('No solution found!')

if __name__ == '__main__':
    main()


In [ ]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def read_distance_matrix(city):
    destination_filepath = f'{city.lower()}_distance_duration_matrices.xlsx'
    df = pd.read_excel(destination_filepath, sheet_name='Distance Matrix')
    df.columns = ['Source', 'Destination', 'Distance (meters)']
    df['Type'] = 'Tourist Attraction'
    
    unique_destinations = pd.unique(df[['Source', 'Destination']].values.ravel('K'))
    destination_index = {destination: index for index, destination in enumerate(unique_destinations)}
    
    distance_matrix = [[0] * len(unique_destinations) for _ in range(len(unique_destinations))]
    
    for _, row in df.iterrows():
        source_index = destination_index[row['Source']]
        dest_index = destination_index[row['Destination']]
        distance_matrix[source_index][dest_index] = row['Distance (meters)']
    
    return distance_matrix, destination_index, unique_destinations

def read_destination_prices(city, df):
    return df['Price'].tolist()

def read_avg_time_spent(city, df):
    return df['Time_Minutes'].tolist()

def create_data_model(city, price, destination_limit_per_day, num_days, num_vehicles=1, depot=0):
    df = pd.read_excel("indonesia_tourism.xlsx", sheet_name="indonesia_tourism")
    distance_matrix, destination_index, unique_destinations = read_distance_matrix(city)
    
    data = {}
    data['distance_matrix'] = distance_matrix
    data['prices'] = read_destination_prices(city, df)
    data['average_spent_time'] = read_avg_time_spent(city, df)
    data['price_threshold'] = price
    data['destination_limit_per_day'] = destination_limit_per_day
    data['num_days'] = num_days
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    data['place_names'] = unique_destinations
    data['destination_index'] = destination_index
    data['city'] = city
    
    return data

def select_hotel_within_budget(city, remaining_budget, hotels):
    valid_hotels = hotels[hotels['price'] <= remaining_budget]
    if valid_hotels.empty:
        return None
    return valid_hotels.loc[valid_hotels['price'].idxmin(), 'name']

def main():
    city = 'Surabaya'
    price_threshold = 2000000
    destination_limit_per_day = 10
    num_days = 1  # Testing with a single day
    
    data = create_data_model(city, price_threshold, destination_limit_per_day, num_days)
    
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    routing = pywrapcp.RoutingModel(manager)
    
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # Price dimension
    def price_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['prices'][from_node]
    
    price_callback_index = routing.RegisterUnaryTransitCallback(price_callback)
    routing.AddDimensionWithVehicleCapacity(
        price_callback_index,
        0,  # no slack
        [data['price_threshold']] * data['num_vehicles'],  # vehicle maximum price capacity
        True,  # start cumul to zero
        'Price')
    
    # Time dimension
    def time_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['average_spent_time'][from_node]
    
    time_callback_index = routing.RegisterTransitCallback(time_callback)
    routing.AddDimensionWithVehicleCapacity(
        time_callback_index,
        0,  # no slack
        [8 * 60] * data['num_vehicles'],  # vehicle maximum time capacity per day (480 minutes)
        True,  # start cumul to zero
        'Time')
    
    def print_solution(manager, routing, solution, data):
        total_distance = 0
        total_price = 0
        total_time = 0
        
        print(f"Day 1 Route:")
        if solution:
            index = routing.Start(0)
            plan_output = f'Route for vehicle 0:\n'
            route_distance = 0
            route_price = 0
            route_time = 0

            remaining_budget = data['price_threshold']
            hotels = pd.read_excel(f'top_{data["city"].lower()}_hotel.xlsx')
            start_hotel = select_hotel_within_budget(data["city"].lower(), remaining_budget, hotels)
            end_hotel = start_hotel

            plan_output += f'Starting Hotel: {start_hotel}\n'
            num_destinations = 0

            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                destination_name = data['place_names'][node_index]

                next_index = solution.Value(routing.NextVar(index))

                # Fetch arc cost for distance
                arc_distance = routing.GetArcCostForVehicle(index, next_index, 0)
                route_distance += arc_distance

                # Fetch arc cost for price
                arc_price = routing.GetArcCostForVehicle(index, next_index, 'Price')
                route_price += arc_price

                # Fetch arc cost for time
                arc_time = routing.GetArcCostForVehicle(index, next_index, 'Time')
                route_time += arc_time

                plan_output += f' {destination_name} -> '
                index = next_index
                num_destinations += 1
            
            plan_output += f' {data["place_names"][manager.IndexToNode(index)]}\n'
            plan_output += f'Ending Hotel: {end_hotel}\n'

            route_distance += 2 * data['distance_matrix'][0][manager.IndexToNode(index)]  # Round trip to depot

            plan_output += f'Distance of the route: {route_distance}m\n'
            plan_output += f'Price of the route: {route_price}\n'
            plan_output += f'Time of the route: {route_time} minutes\n'
            print(plan_output)

            total_distance += route_distance
            total_price += route_price
            total_time += route_time

            print(f"Number of destinations for Day 1: {num_destinations}\n")
        else:
            print(f"No solution found for Day 1!\n")

        print(f'Total Distance of all routes: {total_distance}m')
        print(f'Total Price of all routes: {total_price}')
        print(f'Total Time of all routes: {int(total_time)} minutes')

    
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(search_parameters)
    
    print_solution(manager, routing, solution, data)
    
    # Debug prints
    print("\nDebug Information:")
    print("Number of Nodes:", manager.GetNumberOfNodes())
    print("Number of Vehicles:", data['num_vehicles'])
    print("Price Capacity per Vehicle:", data['price_threshold'])
    print("Time Capacity per Vehicle per Day:", 8 * 60)  # This is per day (480 minutes)
    print("City:", data['city'])

if __name__ == '__main__':
    main()
